# <b> Project 1, Topic 2: Random Constellations and Mythology Generator
### Please access GitHub to use Streamlit, as we cannot add .py files, thank you :)


## By: Xander Graves and Alexis Walker

# <b> Table of Contents:

    Code Cell 1: Imports and Basic Information
    Code Cell 2: Creation of Random Constellations
    Code Cell 3: LLM Mythology Generation
    Code Cell 4: Streamlit

# <b> Code Cell 1: Imports and Basic Information

In [ ]:
import numpy as np
#Used to make values for the stars and clusters, as well as the rotation of the stars (transformation).
import warnings
#Used to ignore unnecessary warnings.
import litellm
#Used to access the LLM, as I mention in the last markdown cell, we used !pip install litellm in order to access the LLM.
import math
import os
#Used to access the astroplot.png file.
import matplotlib.pyplot as plt
#Used to make the graph (random constellations).
from matplotlib.patches import Circle
#Used to make the circle in the graph that we plot the stars in.
import random
from sklearn.cluster import KMeans
#Used to make the clusters of stars, as it was the best clustering algorithm for this project (from our research).
import streamlit as st
#Used to make the Streamlit app.

custom_api_base = "https://litellmproxy.osu-ai.org/"
#OSU's LiteLLM proxy server.

from dotenv import load_dotenv
load_dotenv()
#Loading the .env file.

astro1221_key = os.getenv("ASTRO1221_API_KEY")
#Class API key.
if astro1221_key:
    print("Key found")
else:
    print("Did not find key")
    #Basic check to make sure the API key is working, if not it will print "Did not find key."
    
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")
#Removing unnecessary warnings.

Key found


# <b> Code Cell 2: Creation of Random Constellations

In [ ]:
num_clusters = 5
num_stars = 30
rotation_angle_deg = 0  # degrees; rotate star coords around (0, 0). This will mimic the rotation of the stars in the sky depending on the seasons.
    #0 is winter, 90 is spring, 180 is summer, 270 is fall for this rotation.
#Default values of stars, clusters, and angle, will be changed by Streamlit 

def generate_random_constellations():
    #This is the definition that will help to generate the random constellations.


    fig, ax = plt.subplots(figsize=(8, 8), facecolor="#0e1117")
    ax.set_facecolor("#0e1117")
    global num_stars
    global num_clusters
    global rotation_angle_deg
    #ALlowing num_stars, num_clusters, and rotation_angle_deg to be changed by streamlit
    if num_stars < num_clusters:
        return "Error: Number of clusters is more than the number of stars"
        #Simple check to prevent error from kmeans clustering

    # Use existing base coordinates if available; otherwise create and store them
    if "base_star_coords" in st.session_state:
        base_x_vals, base_y_vals, sizes = st.session_state["base_star_coords"]
        # If the number of stars has changed, regenerate coordinates
        if len(base_x_vals) != num_stars:
            base_x_vals = np.random.uniform(-4, 4, num_stars)
            base_y_vals = np.random.uniform(-3, 3, num_stars)
            sizes = np.random.randint(20, 100, num_stars)
            st.session_state["base_star_coords"] = (base_x_vals, base_y_vals, sizes)
    else:
        base_x_vals = np.random.uniform(-4, 4, num_stars)
        base_y_vals = np.random.uniform(-3, 3, num_stars)
        sizes = np.random.randint(20, 100, num_stars)
        #Creating stars and randomizing their sizes in order to make the stars look more realistic to what they would look like in the sky..
        st.session_state["base_star_coords"] = (base_x_vals, base_y_vals, sizes)

    # Apply rotation to the base coordinates to get the displayed coordinates
    theta = np.radians(rotation_angle_deg)
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    x_vals = base_x_vals * cos_t - base_y_vals * sin_t
    y_vals = base_x_vals * sin_t + base_y_vals * cos_t
    #Using rotation angle from streamlit to rotate the picture

    circle = Circle((0, 0), radius=5, color='black', fill = True)
    ax.add_patch(circle)
    ax.set_aspect('equal', adjustable='datalim')
    #Creating a circle within the graph, stars will only be drawn within the circle.

    coords = np.column_stack((x_vals, y_vals))
    kmeans = KMeans(n_clusters=num_clusters, n_init=10)
    kmeans.fit(coords)
    labels = kmeans.labels_
    #Creating the clusters and assigning labels to each cluster

    for cluster_id in range(kmeans.n_clusters):
        indices = np.where(labels == cluster_id)[0]
        n = len(indices)
        if n < 2:
            continue
        px = x_vals[indices]
        py = y_vals[indices]
        in_tree = np.zeros(n, dtype=bool)
        in_tree[0] = True
        for _ in range(n - 1):
            best_a, best_b, best_d2 = -1, -1, np.inf
            if n < 2:
                continue
            px = x_vals[indices]
            py = y_vals[indices]
            in_tree = np.zeros(n, dtype=bool)
            in_tree[0] = True
            for _ in range(n - 1):
                best_a, best_b, best_d2 = -1, -1, np.inf
                for a in np.where(in_tree)[0]:
                    for b in np.where(~in_tree)[0]:
                        d2 = (px[a] - px[b]) ** 2 + (py[a] - py[b]) ** 2
                        if d2 < best_d2:
                            best_d2, best_a, best_b = d2, a, b
                            #Comparing distances to draw lines, closest distances will have lines drawn to them to give a more realistic constellation look to our code.
                in_tree[best_b] = True
                idx_a, idx_b = indices[best_a], indices[best_b]
                connection = plt.Line2D(
                    [x_vals[idx_a], x_vals[idx_b]],
                    [y_vals[idx_a], y_vals[idx_b]],
                    color="white",
                    alpha=0.6,
                    linewidth=1,
                )
                #The trees are used to make sure that the lines are only plotted within that specific cluster the code is looking at. This is done in order to make sure that no line are made between clusters of stars.
                ax.add_line(connection)
                #Adding the connection to the figure

    ax.scatter(x_vals, y_vals, s=sizes, marker="*", c=kmeans.labels_, alpha=1)
    #Plotting the points on the graph
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_frame_on(False)
    plt.title("Random Constellations", fontsize=14, fontweight="bold", color="white")
    plt.ylim(-5, 5)
    plt.xlim(-5, 5)
    #All cosmetic changes to the graph. This is to make the graph look more like a night sky.
    fig.savefig("astroplot.png")
    #Saving the figure to be used by Streamlit and the LLM

generate_random_constellations()

# <b> Code Cell 3: LLM and Mythology Generation

In [ ]:
import base64

# Opening the file and reading it as binary
with open("astroplot.png", "rb") as f:
    # Reading the image as binary and converting to Base64
    encoded_image = base64.b64encode(f.read()).decode('utf-8')
    

# Prompting the LLM and setting max token values
response = litellm.completion(
    model = "openai/GPT-4.1-mini",
    messages = [
        {"role": "system", "content": "You are a Greek theologian, and have to create a story from the given stars. Use Greek myth stories as a base for these, as a storyteller."},
        #Context for the LLM in order to give proper mythology to the user.
        {"role": "user", "content": [
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{encoded_image}"}}
            #The encoded image being sent to the LLM
        ]}
    ],
    max_tokens = 1000,
    api_base=custom_api_base,
    api_key=astro1221_key,
    temperature=.3
    #LiteLLM Parameters to limit the use of tokens and to make the LLM creative while not going off the rails. The only issue with this is that if you add to many constellations, the LLM will cut off randomly.
    )
print(response)

# <b> Code Cell 4: Streamlit

In [ ]:
def main():
    st.set_page_config(page_title="Constellations & Mythology", layout="wide")
    st.title("Random Constellations Generator with Mythology")
    st.write(
        "This is a constellation generator, where we incorporate LLM into generating mythology with the set of constellations you receive :)")
    #Setting the title and subheading for the Streamlit app

    global num_stars 
    num_stars = st.slider("num_stars", min_value = 1, max_value = 200)
    #Ability to change number of stars via a slider. 200 is the max value of this slider, simulating 200 stars that can be in a cluster (or clusters).

    global num_clusters
    num_clusters = st.slider("num_clusters", min_value = 1, max_value = 20)
    #Giving users the ability to change the amount of clusters. The max value is 20, which is essentially 20 different constellations.

    global rotation_angle_deg
    rotation_angle_deg = st.slider("Rotation (degrees)", min_value=0, max_value=360, value=0)
    #Rotate star coordinates around (0, 0) by this angle before clustering/drawing
    
    if st.button("Generate new constellations"): 
        st.session_state.pop("base_star_coords", None)
        #This deletes the previous base star coordinates, so that the new ones can be generated by the user.
        with st.spinner("Creating constellations..."):
            if num_stars > num_clusters:
                generate_random_constellations()
                st.rerun()
            else: 
                st.warning("Number of clusters exceeds the number of stars. Add more stars or lower the number of clusters.")
                #Same type of warning as in generate_random_constellation, but written in Streamlit

    if st.button("Rotate"):
        #Rotate existing constellation by the current slider angle
        if "base_star_coords" in st.session_state:
            with st.spinner("Rotating..."):
                generate_random_constellations()
            st.rerun()
        else:
            st.warning("Generate constellations first, then use Rotate to spin them.")
            #This is a warning to make sure the user generates a constellation (or constellations) before using the rotation feature.

    if os.path.exists("astroplot.png"):
        st.image("astroplot.png")

        if astro1221_key and st.button("Generate mythology stories"):
            with st.spinner("Asking the LLM for myths..."):
                story = get_mythology_from_llm()
            if story:
                st.subheader("Mythologies")
                st.write(story)
            else:
                st.warning("Could not get mythologies. Check your API key and connection.")
                #If the LLM returns nothing give a warning that the connection of the user may not be working.

    else:
        st.info("Click **Generate new constellations** to create constellations.")
        #This is to loop it back into generating constellations for the user. 


if __name__ == "__main__":
    main()

## Extra Add-Ons: We used !pip install litellm as well as !pip install streamlit in order to access the LLM and the Streamlit App!
<b> Here is our GitHub repository as well: https://github.com/awalker72007/Astro-Coding-1221-Project-1 :)